In [1]:
import netCDF4
import pandas
import datetime
import numpy

In [2]:
minCutoffFitLat = 45.
delTimeCutOffNrstPass = 50 # min
mlonDiffOtrEndCutoff = 50.
delLatCutoff = 2.
delCtimeCutoff = 60. #min

In [3]:
fileList = [ "../poes_n15_20130302_proc.nc",\
                "../poes_n18_20130302_proc.nc",\
                "../poes_m01_20130302_proc.nc",\
                "../poes_n19_20130302_proc.nc",\
                "../poes_m01_20130302_proc.nc",\
                "../poes_n16_20130302_proc.nc",\
                "../poes_n17_20130302_proc.nc" ]
poesAllEleDataDF = pandas.DataFrame( columns =  ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_ele_flux", "sat"] )
poesAllProDataDF = pandas.DataFrame( columns =  ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_pro_flux", "sat"] )
for f in fileList:
    print "reading file-->", f
    # read variable from the netCDF files
    poesRawData = netCDF4.Dataset(f)
    poesDF = pandas.DataFrame( poesRawData.variables['time'][:], columns=[ "timestamp" ] )
    poesDF['date'] = pandas.to_datetime(poesDF['timestamp'], unit='ms')
    poesDF["alt"] = poesRawData.variables['alt'][:]
    poesDF["aacgm_lat_foot"] = poesRawData.variables['aacgm_lat_foot'][:]

    poesDF["aacgm_lon_foot"] = poesRawData.variables['aacgm_lon_foot'][:]
    poesDF["MLT"] = poesRawData.variables['MLT'][:]
    # round of to 2 decimals
    poesDF['alt'] = [ round( x, 2 ) for x in poesDF['alt']]
    poesDF['aacgm_lat_foot'] = [ round( x, 2 ) for x in poesDF['aacgm_lat_foot']]
    poesDF['aacgm_lon_foot'] = [ round( x, 2 ) for x in poesDF['aacgm_lon_foot']]
    poesDF['MLT'] = [ round( x, 2 ) for x in poesDF['MLT']]
    # Add up the fluxes
    poesDF["ted_ele_total_flux"] = poesRawData.variables['ted_ele_tel0_flux_4'][:] +\
            poesRawData.variables['ted_ele_tel0_flux_8'][:] + \
            poesRawData.variables['ted_ele_tel0_flux_11'][:] + \
            poesRawData.variables['ted_ele_tel0_flux_14'][:] + \
            poesRawData.variables['ted_ele_tel30_flux_4'][:] +\
            poesRawData.variables['ted_ele_tel30_flux_8'][:] + \
            poesRawData.variables['ted_ele_tel30_flux_11'][:] + \
            poesRawData.variables['ted_ele_tel30_flux_14'][:]
    poesDF["ted_pro_total_flux"] = poesRawData.variables['ted_pro_tel0_flux_4'][:] +\
            poesRawData.variables['ted_pro_tel0_flux_8'][:] + \
            poesRawData.variables['ted_pro_tel0_flux_11'][:] + \
            poesRawData.variables['ted_pro_tel0_flux_14'][:] + \
            poesRawData.variables['ted_pro_tel30_flux_4'][:] +\
            poesRawData.variables['ted_pro_tel30_flux_8'][:] + \
            poesRawData.variables['ted_pro_tel30_flux_11'][:] + \
            poesRawData.variables['ted_pro_tel30_flux_14'][:]
    poesDF['log_ele_flux'] = [0. if x <= 0. else round( numpy.log10(x), 2 )\
                 for x in poesDF['ted_ele_total_flux']]
    poesDF['log_pro_flux'] = [0. if x <= 0. else round( numpy.log10(x), 2 )\
                 for x in poesDF['ted_pro_total_flux']]
    # the current satellite number
    poesDF["sat"] = f[-19:-17]
#     seperate out electron and proton flux and discard all zeros
    currPoesEleFluxDF = poesDF[poesDF["log_ele_flux"] > 0.][ ["timestamp",\
                     "date", "aacgm_lat_foot", "aacgm_lon_foot", "MLT",\
                     "log_ele_flux", "sat"] ].reset_index(drop=True)
    currPoesProFluxDF = poesDF[poesDF["log_pro_flux"] > 0.][ ["timestamp",\
                     "date", "aacgm_lat_foot", "aacgm_lon_foot", "MLT",\
                     "log_pro_flux", "sat"] ].reset_index(drop=True)
    
    
    poesAllEleDataDF = poesAllEleDataDF.append( currPoesEleFluxDF )
    poesAllProDataDF = poesAllProDataDF.append( currPoesProFluxDF )
    # now delete all the rows for prev DFs
    # we don't want to duplicate data
    poesDF = poesDF.drop( poesDF.index )
    currPoesEleFluxDF = currPoesEleFluxDF.drop( currPoesEleFluxDF.index )
    currPoesProFluxDF = currPoesProFluxDF.drop( currPoesProFluxDF.index )
# create a date and time columns
poesAllEleDataDF["dateStr"] = poesAllEleDataDF["date"].map(lambda x: x.strftime('%Y%m%d'))
poesAllEleDataDF["time"] = poesAllEleDataDF["date"].map(lambda x: x.strftime('%H%M'))
poesAllProDataDF["dateStr"] = poesAllProDataDF["date"].map(lambda x: x.strftime('%Y%m%d'))
poesAllProDataDF["time"] = poesAllProDataDF["date"].map(lambda x: x.strftime('%H%M'))

reading file--> ../poes_n15_20130302_proc.nc
reading file--> ../poes_n18_20130302_proc.nc
reading file--> ../poes_m01_20130302_proc.nc
reading file--> ../poes_n19_20130302_proc.nc
reading file--> ../poes_m01_20130302_proc.nc
reading file--> ../poes_n16_20130302_proc.nc
reading file--> ../poes_n17_20130302_proc.nc


In [4]:
timeRange = [ poesAllEleDataDF["date"].min(), poesAllEleDataDF["date"].max() ]
ctime = timeRange[0]
timeInterval=datetime.timedelta(minutes=30)
while ctime <= timeRange[1]:
    ctime += timeInterval
    if abs( ctime - datetime.datetime(2013,3,2,4) ) < datetime.timedelta(minutes=1):
        break
print ctime

2013-03-02 04:00:00.201000


In [5]:
# We only need those times when POES was above minCutoffFitLat(45) MLAT
poesAllEleDataDF = poesAllEleDataDF[ \
                ( abs( poesAllEleDataDF["aacgm_lat_foot"] ) >= minCutoffFitLat )\
                ].reset_index(drop=True)
# We only need a few columns, discard the rest
poesAllEleDataDF = poesAllEleDataDF[ [ 'sat', 'date',\
                        'aacgm_lat_foot', 'aacgm_lon_foot',\
                            'MLT', 'log_ele_flux' ] ]
poesAllEleDataDF["delCtime"] = abs(poesAllEleDataDF["date"] - ctime)
poesAllEleDataDF["delLatFit"] = abs( abs( poesAllEleDataDF["aacgm_lat_foot"] ) -\
                                    abs( minCutoffFitLat ) )
# We are sorting by sats, dates and lats to pick the nearest time
# when the satellite is between two 45 MLATs
poesAllEleDataDFNth = poesAllEleDataDF[ poesAllEleDataDF["aacgm_lat_foot"]\
                        >= 0. ].sort_values( ['sat', 'date', 'aacgm_lat_foot'],\
                                ascending=True ).reset_index(drop=True).drop_duplicates()
poesAllEleDataDFSth = poesAllEleDataDF[ poesAllEleDataDF["aacgm_lat_foot"]\
                        < 0. ].sort_values( ['sat', 'date', 'aacgm_lat_foot'],\
                                ascending=True ).reset_index(drop=True).drop_duplicates()

In [6]:
# Now we need to pick the satellite path
# which is closest to the selected time.!
# Northern Hemisphere
poesAllEleDataDFNthST = poesAllEleDataDFNth[ \
                            (poesAllEleDataDFNth["date"] \
                                >= ctime-datetime.timedelta(\
                                minutes=delTimeCutOffNrstPass)) &\
                                (poesAllEleDataDFNth["date"] <=\
                                 ctime+datetime.timedelta(\
                                minutes=delTimeCutOffNrstPass))].reset_index(drop=True)
poesAllEleDataDFNthST = poesAllEleDataDFNthST.sort_values(\
                            ["sat","date"], ascending=[True, True]\
                            ).reset_index(drop=True)
# We'll get the the satellite pass which is moving polewards
# Basically percent change in latitudes should be positive
# for a satellite moving polewards (percent change would help
# with the southern hemisphere lcoations.)
poesAllEleDataDFNthST["latRowDiffs"] = poesAllEleDataDFNthST.groupby("sat")[[\
                "aacgm_lat_foot" ] ].pct_change()
poesAllEleDataDFNthST = poesAllEleDataDFNthST[\
                        poesAllEleDataDFNthST["latRowDiffs"] > 0.\
                        ].reset_index(drop=True)
poesAllEleDataDFNthST = poesAllEleDataDFNthST.sort_values(\
                            ["sat", "aacgm_lat_foot","delCtime"]\
                            ).reset_index(drop=True)
# get the start time
selTimeRangeNthDF = poesAllEleDataDFNthST.groupby("sat").first().reset_index()
# Now if the time difference is too large, discard the satellite data
selTimeRangeNthDF = selTimeRangeNthDF[ selTimeRangeNthDF["delCtime"] <= \
                    datetime.timedelta(minutes=delTimeCutOffNrstPass)\
                    ].reset_index()
selTimeRangeNthDF = selTimeRangeNthDF[ ["sat", "date"] ]
selTimeRangeNthDF.columns = [ "sat", "start_time" ] 

# Now get the end times, simply get all times that are
# greater than start time, sort them by date and get
# lowest deLatFit
poesAllEleDataDFNthET = pandas.merge( poesAllEleDataDFNth,\
                            selTimeRangeNthDF, on="sat" )
poesAllEleDataDFNthET = poesAllEleDataDFNthET[ (\
                                poesAllEleDataDFNthET["date"] >=\
                                poesAllEleDataDFNthET["start_time"] ) &\
                                (poesAllEleDataDFNthET["date"] <=\
                                poesAllEleDataDFNthET["start_time"]+datetime.timedelta(\
                                minutes=delTimeCutOffNrstPass)) ].reset_index(drop=True)
poesAllEleDataDFNthET = poesAllEleDataDFNthET.sort_values(\
                            ["sat","date"], ascending=[True, True]\
                            ).reset_index(drop=True)
# We'll get the the satellite pass which is moving equatorwards
# Basically percent change in latitudes should be negative
# for a satellite moving polewards (percent change would help
# with the southern hemisphere lcoations.)
poesAllEleDataDFNthET["latRowDiffs"] = poesAllEleDataDFNthET.groupby("sat")[[\
                "aacgm_lat_foot" ] ].pct_change()
poesAllEleDataDFNthET = poesAllEleDataDFNthET[\
                        poesAllEleDataDFNthET["latRowDiffs"] < 0.\
                        ].reset_index(drop=True)
poesAllEleDataDFNthET = poesAllEleDataDFNthET.sort_values(\
                            ["sat", "aacgm_lat_foot","delCtime"]\
                            ).reset_index(drop=True)
# get the start time
eTimeNthDF = poesAllEleDataDFNthET.groupby("sat").first().reset_index()
eTimeNthDF = eTimeNthDF[ ["sat", "date"] ]
eTimeNthDF.columns = [ "sat", "end_time" ] 
selTimeRangeNthDF = pandas.merge( selTimeRangeNthDF, eTimeNthDF, on="sat" )
print selTimeRangeNthDF

  sat              start_time                end_time
0  01 2013-03-02 03:26:54.755 2013-03-02 03:55:02.752
1  16 2013-03-02 04:28:24.986 2013-03-02 04:55:20.987
2  17 2013-03-02 03:56:16.193 2013-03-02 04:21:50.193
3  18 2013-03-02 03:10:09.132 2013-03-02 03:31:43.132
4  19 2013-03-02 04:42:44.994 2013-03-02 05:08:46.993


In [7]:
# Now we need to pick the satellite path
# which is closest to the selected time.!
# Northern Hemisphere
poesAllEleDataDFSthST = poesAllEleDataDFSth[ \
                            (poesAllEleDataDFSth["date"] \
                                >= ctime-datetime.timedelta(\
                                minutes=delTimeCutOffNrstPass)) &\
                                (poesAllEleDataDFSth["date"] <=\
                                 ctime+datetime.timedelta(\
                                minutes=delTimeCutOffNrstPass))].reset_index(drop=True)
poesAllEleDataDFSthST = poesAllEleDataDFSthST.sort_values(\
                            ["sat","date"], ascending=[True, True]\
                            ).reset_index(drop=True)
# We'll get the the satellite pass which is moving polewards
# Basically percent change in latitudes should be positive
# for a satellite moving polewards (percent change would help
# with the southern hemisphere lcoations.)
poesAllEleDataDFSthST["latRowDiffs"] = poesAllEleDataDFSthST.groupby("sat")[[\
                "aacgm_lat_foot" ] ].pct_change()
poesAllEleDataDFSthST = poesAllEleDataDFSthST[\
                        poesAllEleDataDFSthST["latRowDiffs"] > 0.\
                        ].reset_index(drop=True)
poesAllEleDataDFSthST = poesAllEleDataDFSthST.sort_values(\
                            ["sat", "aacgm_lat_foot","delCtime"],\
                            ascending=[True, False, True]\
                            ).reset_index(drop=True)
# # get the start time
selTimeRangeSthDF = poesAllEleDataDFSthST.groupby("sat").first().reset_index()
# Now if the time difference is too large, discard the satellite data
selTimeRangeSthDF = selTimeRangeSthDF[ selTimeRangeSthDF["delCtime"] <= \
                    datetime.timedelta(minutes=delTimeCutOffNrstPass)\
                    ].reset_index()
selTimeRangeSthDF = selTimeRangeSthDF[ ["sat", "date"] ]
selTimeRangeSthDF.columns = [ "sat", "start_time" ] 


# # Now get the end times, simply get all times that are
# # greater than start time, sort them by date and get
# # lowest deLatFit
poesAllEleDataDFSthET = pandas.merge( poesAllEleDataDFSth,\
                            selTimeRangeSthDF, on="sat" )
poesAllEleDataDFSthET = poesAllEleDataDFSthET[ (\
                                poesAllEleDataDFSthET["date"] >=\
                                poesAllEleDataDFSthET["start_time"] ) &\
                                (poesAllEleDataDFSthET["date"] <=\
                                poesAllEleDataDFSthET["start_time"]+datetime.timedelta(\
                                minutes=delTimeCutOffNrstPass)) ].reset_index(drop=True)
poesAllEleDataDFSthET = poesAllEleDataDFSthET.sort_values(\
                            ["sat","date"], ascending=[True, True]\
                            ).reset_index(drop=True)
# We'll get the the satellite pass which is moving equatorwards
# Basically percent change in latitudes should be negative
# for a satellite moving polewards (percent change would help
# with the southern hemisphere lcoations.)
poesAllEleDataDFSthET["latRowDiffs"] = poesAllEleDataDFSthET.groupby("sat")[[\
                "aacgm_lat_foot" ] ].pct_change()
poesAllEleDataDFSthET = poesAllEleDataDFSthET[\
                        poesAllEleDataDFSthET["latRowDiffs"] < 0.\
                        ].reset_index(drop=True)
poesAllEleDataDFSthET = poesAllEleDataDFSthET.sort_values(\
                            ["sat", "aacgm_lat_foot","delCtime"],\
                            ascending=[True, False, True]\
                            ).reset_index(drop=True)
# get the start time
eTimeSthDF = poesAllEleDataDFSthET.groupby("sat").first().reset_index()
eTimeSthDF = eTimeSthDF[ ["sat", "date"] ]
eTimeSthDF.columns = [ "sat", "end_time" ] 
selTimeRangeSthDF = pandas.merge( selTimeRangeSthDF, eTimeSthDF, on="sat" )
print selTimeRangeSthDF

  sat              start_time                end_time
0  01 2013-03-02 04:17:26.750 2013-03-02 04:43:44.748
1  16 2013-03-02 03:35:52.987 2013-03-02 04:01:26.987
2  17 2013-03-02 04:45:02.194 2013-03-02 05:13:26.194
3  18 2013-03-02 03:58:31.132 2013-03-02 04:26:09.132
4  19 2013-03-02 03:53:02.994 2013-03-02 04:19:56.993


In [8]:
#   sat              start_time                end_time
# 0  01 2013-03-02 03:26:54.755 2013-03-02 03:55:02.752
# 1  16 2013-03-02 04:28:24.986 2013-03-02 04:55:20.987
# 2  17 2013-03-02 03:56:16.193 2013-03-02 04:21:50.193
# 3  18 2013-03-02 03:15:05.133 2013-03-02 03:31:43.132
# 4  19 2013-03-02 04:42:44.994 2013-03-02 05:08:46.993